# Рекомендация тарифов

В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — мы её уже сделали.

**Цель**

Построить модель, которая выберет подходящий тариф, с максимально большим значением *accuracy*. Нужно довести долю правильных ответов по крайней мере до 0.75. Проверить *accuracy* на тестовой выборке самостоятельно.

**План исследования**

* [Открыть и изучить файл](#section_1)
* [Разбить данные на выборки](#section_2)
* [Исследовать модели](#section_3)
* [Проверить модель на тестовой выборке](#section_4)
* [Проверить модель на адекватность](#section_5)
* [Общие выводы](#section_6)

<a id='section_1'></a>
## Открыть и изучить файл

Импортруем необходимые библиотеки

In [4]:
import pandas as pd
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

Читаем файл

In [5]:
df = pd.read_csv('/datasets/users_behavior.csv')

Посмотрим на первые строки

In [89]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


Посмотрим на информацию о столбцах

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Согласно описанию столбцов
Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:
* *сalls* — количество звонков,
* *minutes* — суммарная длительность звонков в минутах,
* *messages* — количество sms-сообщений,
* *mb_used* — израсходованный интернет-трафик в Мб,
* *is_ultra* — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

In [88]:
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


### Выводы 
Файл открылся и прочитался без проблем, столбцы совпадают с заявленными. Пропусков нет, значения адекватны. С этим можно работать

<a id='section_2'></a>
## Разбить данные на выборки

Для начала выделим признаки, на основании которых будет производиться классификация. Это все признаки, кроме `is_ultra`

In [9]:
features = df.drop('is_ultra', axis=1)

Целевой показатель - `is_ultra`, который определяет тариф

In [10]:
target = df['is_ultra']

Отделим сначала тестовую выборку. Размер её определим как 20% от исходного

In [11]:
features_train, features_test, target_train, target_test= train_test_split(features, target, test_size=0.2, random_state = 12345)

Затем из оставшейся выборки `features_train` и `target_train` ещё раз выделим валидационную выборку. Её размер примем как размер тестовой, но с учётом уменьшения размера исходной выборки, параметр `test_size` будет равен 0.2/0.8 = 0.25

In [12]:
features_train, features_valid, target_train, target_valid = train_test_split(features_train, target_train, test_size=0.25, random_state = 12345)

Посмотрим, как разделились данные 

In [13]:
len(features_train), len(features_valid), len(features_test)

(1928, 643, 643)

In [15]:
len(target_train), len(target_valid), len(target_test)

(1928, 643, 643)

### Выводы
Выполнили разделение исходных данных на тренировочную, валидационную и тестовую выборки в соотношении 3:1:1. 

<a id='section_3'></a>
## Исследовать модели

### Дерево решений

Для начала попробуем построить модель на основе *DecisionTreeClassifier*. Для этого будем обучать модель на тренировочной выборке и изменять глубину в цикле от 1 до 10. Сравним *accuracy* на валидациоонной выборке и выберем лучший вариант.

In [45]:
tree_list=[]
for i in range(1,11):
    model = tree.DecisionTreeClassifier(max_depth = i, random_state = 12345)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    result = accuracy_score(target_valid, predictions_valid)
    tree_list.append(result)

In [46]:
tree_list

[0.7387247278382582,
 0.7573872472783826,
 0.7651632970451011,
 0.7636080870917574,
 0.7589424572317263,
 0.7573872472783826,
 0.7744945567651633,
 0.7667185069984448,
 0.7620528771384136,
 0.7713841368584758]

Из этого словаря выберем глубину с лучшей метрикой

In [48]:
max(tree_list)

0.7744945567651633

Определим индекс в списке с лучшей метрикой. Учтём, что индексация в словаре начинается в 0, а глубина с 1.

In [50]:
best_depth = tree_list.index(max(tree_list))+1

In [51]:
best_depth

7

Тогда работаем с моделью с глубиной 7

In [56]:
best_tree_model = tree.DecisionTreeClassifier(max_depth = best_depth, random_state = 12345)
best_tree_model.fit(features_train, target_train)
best_tree_model.score(features_valid, target_valid)

0.7744945567651633

In [85]:
best_tree_model.get_params()

{'class_weight': None,
 'criterion': 'gini',
 'max_depth': 7,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'presort': False,
 'random_state': 12345,
 'splitter': 'best'}

### Случайный лес

Построим модель на основании *RandomForestClassifier*. Для этого будем обучать модель на тренировочной выборке и изменять количество деревьев от 0 до 10. Сравним accuracy на валидационной выборке и выберем лучший вариант.


In [58]:
forest_list = []
for i in range(1,11):
    model = RandomForestClassifier(random_state = 12345, n_estimators = i)
    model.fit(features_train, target_train)
    result = model.score(features_valid, target_valid)
    forest_list.append(result)

In [59]:
forest_list

[0.702954898911353,
 0.7573872472783826,
 0.744945567651633,
 0.7651632970451011,
 0.7620528771384136,
 0.7698289269051322,
 0.7713841368584758,
 0.7869362363919129,
 0.7838258164852255,
 0.7884914463452566]

Определим индекс в списке с лучшей метрикой. Учтём, что индексация в словаре начинается в 0, а параметр с 1.

In [61]:
best_estimators = forest_list.index(max(forest_list))+1

In [62]:
best_estimators

10

Для дальнейшей работы будем использовать модель с параметром n_estimators = 10. В следующем цикле посмотрим, как метрика зависит от глубины. Можно было организовать вложенный цикл по глубине в первом, но решил что раздельными циклами проще.

In [86]:
forest_list = []
for i in range(1,11):
    model = RandomForestClassifier(random_state = 12345, n_estimators = best_estimators, max_depth = i)
    model.fit(features_train, target_train)
    result = model.score(features_valid, target_valid)
    forest_list.append(result)

In [87]:
forest_list

[0.7309486780715396,
 0.7636080870917574,
 0.7713841368584758,
 0.7776049766718507,
 0.7776049766718507,
 0.7853810264385692,
 0.7869362363919129,
 0.7853810264385692,
 0.7947122861586314,
 0.7900466562986003]

Определим индекс в списке с лучшей метрикой. Учтём, что индексация в словаре начинается в 0, а параметр с 1.

In [65]:
best_forest_depth = forest_list.index(max(forest_list))+1

In [66]:
best_forest_depth

9

Таким образом, лучшей моделью будет модель леса с к-вом деревьев 10 и глубиной 9.

In [68]:
best_forest_model = RandomForestClassifier(random_state = 12345, n_estimators = best_estimators, max_depth = best_forest_depth)
best_forest_model.fit(features_train, target_train)
best_forest_model.score(features_valid, target_valid)

0.7947122861586314

In [69]:
best_forest_model.get_params()

{'bootstrap': True,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': 9,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 10,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 12345,
 'verbose': 0,
 'warm_start': False}

### Логистическая регрессия

Построим модель на основании *LogisticRegression*.

In [120]:
logistic_model = LogisticRegression(C= 0.5, random_state = 12345)
logistic_model.fit(features_train, target_train)
result = logistic_model.score(features_valid, target_valid)
print(result)

0.7293934681181959


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [125]:
logistic_model.get_params()

{'C': 0.5,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'warn',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': 12345,
 'solver': 'warn',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

Модель на логистической регрессии показала недопустимую *accuracy*

### Выводы

Были исследованы 3 модели:

* DecisionTreeClassifier
* RandomForestClassifier
* LogisticRegression

Для первых двух выбраны оптимальные гиперпараметры. При этом глубина дерева решения оптимальная - 7, а к-во деревьев в случайном лесe - 10. На валидационной выборке эти модели показали *accuracy* выше заданной. Модель логистической регрессии показала слишком низкую *accuracy*.

<a id='section_4'></a>
## Проверить модель на тестовой выборке

Проверим значение метрики для использованных моделей.

### Дерево решений

In [126]:
best_tree_model.score(features_test, target_test)

0.7884914463452566

### Случайный лес

In [127]:
best_forest_model.score(features_test, target_test)

0.7947122861586314

### Логистическая регрессия

In [128]:
logistic_model.score(features_test, target_test)

0.7527216174183515

### Выводы.
На тестовой выборке все модели показали достаточный результат. Однако лучший результат выдал случайный лес. Эту модель можем рекомендовать для использования.

<a id='section_5'></a>
## Проверить модели на адекватность

Для проверки на адекватность сравним полученные ранее результаты с моделью *DummyClassifier*. Для того, чтобы выбрать лучшую стратегию посмотрим, как распределены тарифы в исходных данных.

In [107]:
df['is_ultra'].mean()

0.30647168637212197

Значение порядка 0.3. Это говорит о том, что записей с тарифом "Смарт" (значение 0) больше.

In [108]:
df['is_ultra'].value_counts()

0    2229
1     985
Name: is_ultra, dtype: int64

Создадим модель с параметром стратегии - наиболее частый. 

In [109]:
from sklearn.dummy import DummyClassifier
dummy_model = DummyClassifier(strategy = 'most_frequent')

Обучим на тестовой выборке

In [110]:
dummy_model.fit(features_train, target_train)

DummyClassifier(constant=None, random_state=None, strategy='most_frequent')

Посмотрим на результат (*accuracy*) на валидационной выборке

In [130]:
dummy_model.score(features_valid, target_valid)

0.6889580093312597

И на тестовой

In [129]:
dummy_model.score(features_test, target_test)

0.6951788491446346

Видим, что эта модель имеет метрику ниже, чем модели дерева, леса и регрессии. 

### Выводы

Модель-манекен имеет метрику ниже, чем рассмотренные модели дерева решений, случайного леса и логистической регрессии. Это говорит о том, что модели адекватны, так как результат у них выше.

<a id='section_6'></a>
## Общие выводы

В ходе исследования изучали данные о тарифах и построили модель, которая выберает подходящий тариф, с максимально большим значением *accuracy*. Исходные данные разделили на тренировочную, валидационную и тестовую выборки. Исследовали 3 модели :
* DecisionTreeClassifier
* RandomForestClassifier
* LogisticRegression

На тестовой выборке все модели показали результат лучше заданного (>0.75). Однако наилучший результат показало дерево решений (0.788). При проверке на адекватность установили, что все модели имееют *accuracy* больше, чем модель-манекен. 